In [1]:
import numpy as np
from collections.abc import Iterable
import itertools
from itertools import chain, combinations

In [2]:
# random partition of data into A, B, C data owners
def partition(data, num, num_players):
    split_points = np.random.choice(num - 1, num_players - 1, replace=False) + 1
    split_points.sort()
    div = list(np.split(data, split_points))
    if (pseudoShap(data, div[0],div[2])>pseudoShap(data, div[1],div[2])): # fulfilling precondition that S(A) > S(B)
        return div
    else:
        return partition(data, num, num_players)

In [3]:
# used for creating all possible subsets of A data to shift over to B data
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

In [4]:
# utility of a coalition - number of data points (each 1) + max replacement utility of missing data
def utility(full_data, data):
    missing = list(set(full_data)-set(data))
    return len(data)+findMax(sim_matrix,data,missing)

In [5]:
# finding the max replacement utility of missing data
def findMax(similarity_matrix, have, not_have):
    s = 0
    for j in not_have:
        arr=[]
        for i in have:
            arr.append(similarity_matrix[i-1][j-1])
        if (len(arr)>0):
            s+=max(arr)
    return s

In [6]:
# pseudo-Shapley value for our "two player" game, for ex. U(B union C) - U(C) where C is the base data/algorithm
def pseudoShap(all_data, target, base_data):
    return (utility(all_data, list(target)+list(base_data))-utility(all_data,base_data))

In [7]:
# helper function for flattening array of arrays
def flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

In [8]:
n_data = 10 # total number of data points
n_owners = 3 # number of data owners, C is the "base" data
full_data = range(1,n_data) # full set of data
sim_matrix = np.random.random((n_data-1, n_data-1)) # replacement utility matrix (similarity matrix)
np.fill_diagonal(sim_matrix, 0) # filling diagonals with 0 because a data entry cannot be used to replace itself (would mess up max replacement calculations)

In [9]:
division = partition(full_data, n_data, n_owners) # randomly dividing the data amongst 3 owners
#to_shift = list(powerset(division[0])) # list of all possible subsets from A to shift
to_shift = list(itertools.combinations(division[0] , 1)) # list of all single tuples from A to shift

In [10]:
print("Starting utilities and Shapley values:")
print("U(A): "+ str(utility(full_data, list(division[0]))))
print("U(B): "+ str(utility(full_data, list(division[1]))))
print("S(A): "+ str(pseudoShap(full_data,list(division[0]),list(division[2]))))
print("S(B): "+ str(pseudoShap(full_data,list(division[1]),list(division[2]))))

Starting utilities and Shapley values:
U(A): 8.199463702666925
U(B): 6.520879947274746
S(A): 0.7175797353001894
S(B): 0.6528272405761104


In [11]:
def shiftShapCalc(full, div_list, shift_tuples):
    A = list(div_list[0])
    B = list(div_list[1])
    C = list(div_list[2])
    utilA = []
    utilB = []
    shapA = []
    shapB = []
    for i in shift_tuples:
        newA = list(set(A)-set(i))
        newB = list(np.append(B, list(i)))
        utilA.append(utility(full, newA+C)) # more interested in just newA or newA+C?
        utilB.append(utility(full, newB+C)) # more interested in just newB or newB+C?
        shapA.append(utility(full, newA+C)-utility(full,C))
        shapB.append(utility(full, newB+C)-utility(full,C))
    return [utilA, utilB, shapA, shapB]

In [12]:
results = shiftShapCalc(full_data, division, to_shift)
switch_tuples = [] # the tuples of shifted data that switch S(A) > S(B) to S(A) <= S(B)
for i in range(0,len(to_shift)):
    if (results[2][i]<=results[3][i]): # checking for the switch in Shapley Value inequality
        switch_tuples.append(to_shift[i])

In [13]:
switch_tuples

[(1,), (2,), (3,)]